# 📚 Latent Dirichlet Allocation via Stochastic Variational Inference

_Implementation and application of the algorithm proposed in "Stochastic Variational Inference" (Hoffman et al., 2013) for topic modeling on a Wikipedia-based corpus._

## 🧭 Introduction

This project is based on the seminal paper [_Stochastic Variational Inference_](https://www.jmlr.org/papers/volume14/hoffman13a/hoffman13a.pdf) by Hoffman, Blei, Wang and Paisley (2013), which introduces a scalable alternative to classical variational inference by leveraging stochastic optimization techniques.

Our goal is to implement from scratch the algorithm proposed in the article and apply it to a real-world problem: unsupervised topic discovery from a corpus of Wikipedia articles. We focus on **Latent Dirichlet Allocation (LDA)** as the probabilistic model to which we apply **Stochastic Variational Inference (SVI)**.

This notebook will walk through the theoretical foundations, the practical implementation, and the analysis of the resulting topics.

## 🧱 Notebook Structure

1. [Overview of Variational Inference](#variational)
2. [Motivation for Stochastic Variational Inference](#svi)
3. [Summary of the Algorithm in the Article](#algo-summary)
4. [Our Approach and Data Preparation](#our-approach)
5. [Latent Dirichlet Allocation (LDA)](#lda)
6. [SVI Algorithm for LDA](#svi-lda)
7. [Model Training and Hyperparameter Choices](#training)
8. [Results Visualization and Interpretation](#results)
9. [Discussion](#discussion)
10. [Conclusion](#conclusion)

## 📘 Overview of Variational Inference

Variational Inference (VI) is a deterministic technique used to approximate intractable posterior distributions in Bayesian models. Rather than using sampling (like MCMC), VI turns inference into an optimization problem: it seeks the closest distribution (within a tractable family) to the true posterior by minimizing the Kullback–Leibler (KL) divergence.

Mathematically, given a model with observed variables $x$ and latent variables $z$, and a prior $p(z)$, VI introduces a variational distribution $q(z; \lambda)$ and minimizes:

$$
\text{KL}(q(z; \lambda) \| p(z \mid x)) \quad \Longleftrightarrow \quad \text{maximize } \mathcal{L}(q) = \mathbb{E}_q[\log p(x, z)] - \mathbb{E}_q[\log q(z)]
$$

This objective, $\mathcal{L}(q)$, is known as the **Evidence Lower Bound (ELBO)**. It's maximized when $q(z) \approx p(z \mid x)$.

**Applications**:
- Topic modeling (LDA)
- Bayesian neural networks
- Hidden Markov Models
- Probabilistic matrix factorization

## ⚡ Why Stochastic Variational Inference?

Classical variational inference methods require full passes over the data to optimize the ELBO, making them computationally expensive and non-scalable for large datasets.

Hoffman et al. (2013) propose **Stochastic Variational Inference (SVI)** as a solution to this issue. SVI leverages mini-batch optimization and stochastic gradients to update the global variational parameters using small subsets of data.

This makes it particularly suited to:
- Streaming or online learning
- Large-scale text corpora
- Models with complex posterior structures

SVI has since become foundational in scalable Bayesian inference and is widely adopted in modern probabilistic programming libraries (e.g., Pyro, TensorFlow Probability).

## 🔁 Summary of the Algorithm from Hoffman et al. (2013)

The algorithm alternates between:
1. **Local inference**: Estimating document-specific variational parameters $\phi$, $\gamma$
2. **Global update**: Updating the global parameters $\lambda$ via a learning rate $\rho_t$

Each iteration samples a document and performs:
- An E-step: variational updates of $\phi$ and $\gamma$
- An M-step: update of $\lambda$ via:

$$
\lambda^{(t)} = (1 - \rho_t)\lambda^{(t-1)} + \rho_t \hat{\lambda}
$$

Where $\rho_t = (\tau + t)^{-\kappa}$ is the learning rate and $\hat{\lambda}$ is a scaled intermediate estimate based on the sampled document.

This stochastic approximation converges under mild conditions and enables inference on massive datasets.

## 🧩 Our Setup and Corpus

To apply the SVI algorithm to a concrete task, we chose **Latent Dirichlet Allocation (LDA)** on a corpus of **~1200 Wikipedia articles**, structured by 8 distinct themes (seeds) such as History, Art, Science, and Politics.

Each article was fetched using the `wikipedia-api` library, then preprocessed:
- Tokenization and normalization
- Stopword removal
- Vocabulary construction
- Bag-of-Words encoding

This yields a corpus ready for topic modeling.

## 📚 Latent Dirichlet Allocation (LDA)

LDA is a generative probabilistic model where:
- Each document is modeled as a mixture of topics
- Each topic is a distribution over words

### Generative process:
1. For each topic $k$, draw $\beta_k \sim \text{Dir}(\eta)$
2. For each document $d$:
    - Draw $\theta_d \sim \text{Dir}(\alpha)$
    - For each word $n$:
        - Draw topic assignment $z_{dn} \sim \text{Mult}(\theta_d)$
        - Draw word $w_{dn} \sim \text{Mult}(\beta_{z_{dn}})$

This model is widely used for unsupervised discovery of thematic structures in text corpora.

We model $\theta$ and $\beta$ using variational approximations with Dirichlet distributions.

## 🛠️ SVI for LDA — Our Implementation

We implement the algorithm following the exact procedure from Hoffman et al.:
- Initialization of global parameters $\lambda \sim \text{Gamma}$
- Per-document E-step: update $\phi$, $\gamma$
- Global M-step: update $\lambda$ via a decaying learning rate

Our implementation is modular and cleanly separates core steps.

```python
# The function SVI_for_LDA(...) is inserted here


---

### 🟩 **Training and Parameter Choices** <a name="training"></a>

```markdown
## ⚙️ Training and Hyperparameter Choices

To train the model, we used the following configuration:

- $K = 10$ topics
- $\alpha = 1.0$ (Dirichlet prior for document-topic)
- $\eta = 1.0$ (Dirichlet prior for topic-word)
- $\tau = 100$, $\kappa = 0.7$ (controls learning rate decay)
- $max\_iter = 500$, $e\_step\_iter = 30$

These values were chosen based on:
- Recommendations from the original paper
- Common defaults in the literature
- Preliminary testing on the dataset size (~1200 documents)

```python
# Insert SVI model training cell here


---

### 🟩 **Visualizing and Analyzing the Results** <a name="results"></a>

## 📊 Results and Visualizations

We analyze three key aspects of the learned model:
1. __Topic-word distributions__ — the top words most strongly associated with each topic.
2. **Document-topic distributions** — how each article is distributed over topics.
3. **Seed-topic associations** — how the original seed themes align with the inferred topics.

Visualizations include:
- Word clouds per topic
- Pie chart of dominant topic frequencies
- Stacked bar chart showing topic distribution by seed
- Heatmaps of topic-seed associations

## 💬 Discussion

The results show clear patterns of topic specialization across the corpus. Some topics align closely with expected themes (e.g., a topic with "musée", "peinture", "artiste" aligns well with the 'Art' seed).

However, a few topics dominate disproportionately, which suggests:
- Possible imbalance in the corpus
- Overlapping vocabulary between themes
- A need to tune $\alpha$, $K$, or consider topic coherence metrics

Further experiments (e.g., increasing topic count, or using seed-supervision) could help refine topic separation and interpretability.

## 🧾 Conclusion

This project demonstrated a full implementation of **Stochastic Variational Inference** as proposed in Hoffman et al. (2013), applied to unsupervised topic modeling with LDA.

We:
- Built a Wikipedia corpus of ~1200 articles across 8 themes
- Designed and implemented a custom SVI algorithm
- Trained and analyzed an interpretable LDA model
- Visualized the discovered structure across documents and themes

### 🔍 Improvements & Extensions:
- Try dynamic topic models (DTM)
- Explore topic coherence metrics (`c_v`, `NPMI`)
- Use more advanced corpora and larger datasets
- Evaluate document clustering by topic assignments

**SVI remains a powerful and scalable method** for modern probabilistic modeling, and this project showcases its potential for interpretable NLP pipelines.

In [ ]:
import json
import time

import numpy as np
from scipy.special import digamma

from src import *

In [ ]:
with open("data/wikipedia_corpus.json", "r", encoding="utf-8") as f:
    wiki_articles = json.load(f)

texts = [article["content"] for article in wiki_articles.values()] # Extract the content of the articles
titles = [article["title"] for article in wiki_articles.values()] # Extracrt the title title of the articles
corpus, vocab = vectorize_texts(texts) # Vectorized texts

V = len(vocab) # Vocabulary size

In [ ]:
def log_dirichlet_expectation(alpha):
    """
    Computes the expectation of the log of a Dirichlet distribution.

    Args:
        alpha (np.ndarray): Dirichlet parameters, 1D or 2D array.

    Returns:
        np.ndarray: Expected log values of the Dirichlet-distributed variables.
    """
    if len(alpha.shape) == 1:
        return digamma(alpha) - digamma(np.sum(alpha))
    return digamma(alpha) - digamma(np.sum(alpha, axis=1))[:, np.newaxis]

In [ ]:
def SVI_for_LDA(documents, V, K=10, alpha=1.0, eta=1.0, max_iter=100, tau=64, kappa=0.7, e_step_iter=20, verbose=2):
    """
    Trains a Latent Dirichlet Allocation (LDA) model using Stochastic Variational Inference (SVI).

    Args:
        documents (list of list of int): Corpus as lists of word indices.
        V (int): Vocabulary size.
        K (int, optional): Number of topics. Defaults to 10.
        alpha (float, optional): Dirichlet prior for document-topic distribution. Defaults to 1.0.
        eta (float, optional): Dirichlet prior for topic-word distribution. Defaults to 1.0.
        max_iter (int, optional): Number of training iterations. Defaults to 100.
        tau (float, optional): Learning rate delay. Defaults to 64.
        kappa (float, optional): Learning rate decay factor (0.5 < kappa ≤ 1). Defaults to 0.7.
        e_step_iter (int, optional): Number of E-step iterations per document. Defaults to 20.
        verbose (int, optional): Verbosity level (0=silent, 1=summary, 2=full). Defaults to 2.

    Returns:
        tuple:
            - np.ndarray: Learned topic-word distribution matrix (lambda).
            - dict: Mapping from document titles to topic distributions.
    """
    D = len(documents)
    lambd = np.random.gamma(100., 1./100., size=(K, V))

    doc_topic_distrib = {}

    start_training_time = time.time() 
    for t in range(max_iter):
        if verbose >= 2:
            start_iteration_time = time.time() 

        doc_id = np.random.randint(0, D)
        doc = documents[doc_id]
        N = len(doc)

        gamma = np.ones(K)
        phi = np.full(shape=(N, K), fill_value=1/K)

        E_logbeta = log_dirichlet_expectation(lambd)
        exp_E_logbeta = np.exp(E_logbeta)

        for _ in range(e_step_iter):
            E_logtheta = log_dirichlet_expectation(gamma)
            exp_E_logtheta = np.exp(E_logtheta)

            for n, w in enumerate(doc):
                phi[n, :] = exp_E_logtheta * exp_E_logbeta[:, w]
                phi[n, :] /= np.sum(phi[n, :])
            
            gamma = alpha + np.sum(phi, axis=0)

            theta = gamma / np.sum(gamma)
            doc_topic_distrib[titles[doc_id]] = theta

        topic_word_contrib = np.zeros(shape=(K, V))
        for n, w in enumerate(doc):
            topic_word_contrib[:, w] += phi[n, :]

        lambd_hat = eta + D*topic_word_contrib

        rho = (t + tau) ** -kappa
        lambd = (1 - rho) * lambd + rho * lambd_hat

        if verbose >= 2:
            iteration_time = time.time() - start_iteration_time
            print(f"\t - Iteration {t} done in {iteration_time:.2f}s.")

    if verbose >= 1:
        training_time = time.time() - start_training_time
        print(f"Training done in {training_time:.2f}s.")

    return lambd, doc_topic_distrib

In [ ]:
lambd, doc_topic_distrib = SVI_for_LDA(
    corpus,
    V,
    K=10,
    alpha=1.0,
    eta=1.0,
    max_iter=500,
    tau=100,
    kappa=0.7,
    e_step_iter=30,
    verbose=1
)

In [ ]:
top_words = get_top_words(lambd, vocab)
top_words

In [ ]:
plot_top_words_wordcloud(lambd, vocab, n_wordcloud=10)

In [ ]:
docs_info = get_topic_distrib_info(doc_topic_distrib, wiki_articles)
docs_info

In [ ]:
plot_topics_distrib(docs_info)

In [ ]:
topics_by_seed = plot_topics_distrib_by_seed(docs_info)
topics_by_seed